Determine the anchor points of primary forest percentage in 1492, 1697, 1804, and 1921 for hindcasting

In [27]:
import numpy as np
import os
from os.path import join
import sys
from osgeo import gdal_array, gdal, gdalconst

pwd = os.getcwd()
rootpath = os.path.abspath(os.path.join(pwd, '../..'))
path_pythoncode = join(rootpath, 'pythoncode')
sys.path.append(path_pythoncode)

from land_change_model_publish_lc.change_matrix_published_version import land_cover_map_read_published_version

if __name__ == '__main__':

    filename_country_id = join(rootpath, 'data', 'shapefile', 'landmask', 'countryid_hispaniola.tif')
    img_country_id = gdal_array.LoadFile(filename_country_id)

    img_central_valley = gdal_array.LoadFile(join(rootpath, 'data', 'historical_records', '1697_Central Valley_AG', '1697_central_valley_ag.tif'))

    filename_dem = join(rootpath, 'data', 'dem', 'hispaniola_dem_info', 'dem_mosaic.tif')
    img_dem = gdal_array.LoadFile(filename_dem)
    img_dem = img_dem.astype(float)
    img_dem[img_country_id == 0] = np.nan
    
    # read the land cover in 1996 and get the mask of PF in 1996
    img_lc_1996 = land_cover_map_read_published_version(year=1996, country_flag='hispaniola')
    mask_pf_1996 = (img_lc_1996 == 2) | (img_lc_1996 == 3)
    
    count_land_total = np.count_nonzero(img_country_id > 0)
    # water and wetland keep stable in all years 
    count_land_exclude_water_wetland = count_land_total - np.count_nonzero((img_lc_1996 == 6) | (img_lc_1996 == 9))
    print('count_land_total: ', count_land_total)
    print('count_land_exclude_water_wetland: ', count_land_exclude_water_wetland)
    
    # primary forest target in 1492: 96% of Hispaniola is primary forest, rest 4% is agriculture
    pf_count_1492 = int(np.ceil(count_land_exclude_water_wetland * 0.96))
    print(f'percentage of PF in 1492: {pf_count_1492 / count_land_total}')

count_land_total:  83554703
count_land_exclude_water_wetland:  82910737
percentage of PF in 1492: 0.9526011719531814


In [34]:
    np.count_nonzero((img_lc_1996 == 3) & (img_country_id == 2)) / np.count_nonzero(img_country_id == 2)

0.011383742132023165

In [29]:
    # primary forest target in 1697: Central valley did not have PF, above 600 meter is PF, below 300 meter is not PF, 
    # between 300-600 meter is not sure, used to determine the boundary 
    # primary dry forest below 300 meter should be considered and adjusted
    mask_elevation_1697 = (img_country_id > 0) & (img_dem > 300) & (img_central_valley == 0)
    # pf_count_1697 = np.count_nonzero(mask_elevation_1697) + np.count_nonzero(mask_pf_1996 & ~mask_elevation_1697)
    
    # Minimum PF in 1697: using 600 meter as the threshold
    proportion_pf_above_600 = np.count_nonzero((mask_pf_1996 & (img_dem > 600))) / np.count_nonzero(img_dem > 600)  # proportion of primary forest above 600 meter
    proportion_pf_below_600 = np.count_nonzero((mask_pf_1996 & (img_dem <= 600))) / np.count_nonzero(img_dem <= 600)
    
    print(f'proportion of PF above 600 meter in 1996: {proportion_pf_above_600}')
    print(f'proportion of PF below 600 meter in 1996: {proportion_pf_below_600}')
    
    # adjust the primary forest count below 600 meter based on the proportion of primary forest above 600 meter
    # if in 1697, below 600 meter is 100%, then the PF below 600 meter should be adjusted, PF_count_below_600 / proportion_pf_above_600
    adjust_pf_below_600 = np.count_nonzero((mask_pf_1996 & (img_dem <= 600))) / proportion_pf_above_600 
    
    # The final PF count in 1697 = PF count above 600 meter + adjusted PF count below 600 meter
    pf_count_minimum_1697 = np.count_nonzero((img_country_id > 0) & (img_dem > 600) & (img_central_valley == 0)) + adjust_pf_below_600
    pf_count_minimum_1697 = int(np.ceil(pf_count_minimum_1697))
    
    print(f'minimum percentage of PF in 1697: {pf_count_minimum_1697 / count_land_total}')

proportion of PF above 600 meter in 1996: 0.17241173304809088
proportion of PF below 600 meter in 1996: 0.010128036935816853
minimum percentage of PF in 1697: 0.28039154181422915


In [30]:
    # Maximum in 1697: using 300 meter as the threshold
    proportion_pf_above_300 = np.count_nonzero((mask_pf_1996 & (img_dem > 300))) / np.count_nonzero(img_dem > 300)  # proportion of primary forest above 600 meter
    proportion_pf_below_300 = np.count_nonzero((mask_pf_1996 & (img_dem <= 300))) / np.count_nonzero(img_dem <= 300)
    
    print(f'proportion of PF above 300 meter in 1996: {proportion_pf_above_300}')
    print(f'proportion of PF below 300 meter in 1996: {proportion_pf_below_300}')
    
    # adjust the primary forest count below 600 meter based on the proportion of primary forest above 600 meter
    # if in 1697, below 600 meter is 100%, then the PF below 600 meter should be adjusted, PF_count_below_600 / proportion_pf_above_600
    adjust_pf_below_300 = np.count_nonzero((mask_pf_1996 & (img_dem <= 300))) / proportion_pf_above_300 
    
    # The final PF count in 1697 = PF count above 600 meter + adjusted PF count below 600 meter
    pf_count_maximum_1697 = np.count_nonzero((img_country_id > 0) & (img_dem > 300) & (img_central_valley == 0)) + adjust_pf_below_300
    pf_count_maximum_1697 = int(np.ceil(pf_count_maximum_1697))
    
    print(f'maximum percentage of PF in 1697: {pf_count_maximum_1697 / count_land_total}')
    

proportion of PF above 300 meter in 1996: 0.09352127655098554
proportion of PF below 300 meter in 1996: 0.012745935961893457
maximum percentage of PF in 1697: 0.4852002406136253


In [31]:
    # primary forest target in 1804: all of Haiti above 1000 meter and all of DR above 600 meter are primary forest
    mask_elevation_1804 = ((img_country_id == 1) & (img_dem > 1000)) | ((img_country_id == 2) & (img_dem > 600))
    pf_count_1804 = np.count_nonzero(mask_elevation_1804) + np.count_nonzero((img_lc_1996 == 3) & ~mask_elevation_1804)
    
    proportion_pf_haiti_above_1000 = np.count_nonzero(mask_pf_1996 & (img_dem > 1000)) / np.count_nonzero(img_dem > 1000)
    proportion_pf_haiti_below_1000 = np.count_nonzero(mask_pf_1996 & (img_dem <= 1000)) / np.count_nonzero(img_dem <= 1000)
    
    print(f'proportion of PF above 1000 meter in 1996, Haiti: {proportion_pf_haiti_above_1000}')
    print(f'proportion of PF below 1000 meter in 1996, Haiti: {proportion_pf_haiti_below_1000}')
    
    proportion_pf_dr_above_600 = np.count_nonzero(mask_pf_1996 & (img_dem > 600)) / np.count_nonzero(img_dem > 600)
    proportion_pf_dr_below_600 = np.count_nonzero(mask_pf_1996 & (img_dem <= 600)) / np.count_nonzero(img_dem <= 600)
    
    print(f'proportion of PF above 600 meter in 1996, DR: {proportion_pf_dr_above_600}')
    print(f'proportion of PF below 600 meter in 1996, DR: {proportion_pf_dr_below_600}')
    
    adjust_pf_haiti_below_1000 = np.count_nonzero(mask_pf_1996 & (img_dem <= 1000)) / proportion_pf_haiti_above_1000
    adjust_pf_dr_below_600 = np.count_nonzero(mask_pf_1996 & (img_dem <= 600)) / proportion_pf_dr_above_600
    
    pf_count_1804 = np.count_nonzero((img_country_id == 1) & (img_dem > 1000)) + adjust_pf_haiti_below_1000 + np.count_nonzero((img_country_id == 2) & (img_dem > 600)) + adjust_pf_dr_below_600
    pf_count_1804 = int(np.ceil(pf_count_1804))
    
    print(f'percentage of PF in 1804: {pf_count_1804 / count_land_total}')    

proportion of PF above 1000 meter in 1996, Haiti: 0.34980413832611834
proportion of PF below 1000 meter in 1996, Haiti: 0.012254709442410205
proportion of PF above 600 meter in 1996, DR: 0.17241173304809088
proportion of PF below 600 meter in 1996, DR: 0.010128036935816853
percentage of PF in 1804: 0.25190558094617366


In [32]:
    # primary forest increase target in 1921: 5%-10% in Haiti and 10%-30% in DR
    pf_count_1921 = np.count_nonzero(img_country_id == 1) * 0.075 + np.count_nonzero(img_country_id == 2) * 0.2
    pf_count_1921 = int(np.ceil(pf_count_1921))
    
    pf_count_minimum_1921 = np.count_nonzero(img_country_id == 1) * 0.05 + np.count_nonzero(img_country_id == 2) * 0.1
    pf_count_maximum_1921 = np.count_nonzero(img_country_id == 1) * 0.1 + np.count_nonzero(img_country_id == 2) * 0.3
    
    print(f'percentage of PF in 1921: {pf_count_1921 / count_land_total}')
    print(f'minimum percentage of PF in 1921: {pf_count_minimum_1921 / count_land_total}')
    print(f'maximum percentage of PF in 1921: {pf_count_maximum_1921 / count_land_total}')

percentage of PF in 1921: 0.1549410330618972
minimum percentage of PF in 1921: 0.08197641190825608
maximum percentage of PF in 1921: 0.22790564763302432
